In [ ]:
filtname = "i"
objname = "yasone2"
objid = "01"
stdid = "11"

filename = f"../{objname}/img_{filtname}_{objid}/nobkg.fits"
stdname = f"../std1/img_{filtname}_{stdid}/flat_fielded-astrom-zeropoint.toml"


In [ ]:
bkg_rms = CCDData.read(filename.replace("nobkg", "bkg_err"))

In [ ]:
sigma5_depth = {
    "r": 24.55,
    "i": 24.03,
    "g": 25.03
}[filtname]

In [ ]:
ls ../yasone2/img_g_01

In [ ]:
%matplotlib ipympl

In [ ]:
ls ../std1/img_g_11

In [ ]:
import sep
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord

import tomllib

In [ ]:

import sys
sys.path.append("../../imaging")
sys.path.append("..")

from convenience_functions import show_image
from phot_utils import plot_sources, swap_byteorder, xmatch, to_mag, get_atm_extinction, plate_scale


In [ ]:
from astropy.table import Table

In [ ]:
import arya

In [ ]:
img = CCDData.read(filename)

In [ ]:
exptime = img.header["exptime"]
gain = img.header["gain"]

In [ ]:
with open(stdname, "rb") as f:
    std_data = tomllib.load(f)
std_data

In [ ]:
assert std_data["filter"] == img.header["filter2"]

In [ ]:
atm_ext, atm_ext_err = get_atm_extinction(img.header["airmass"], img.header["filter2"])

# Sky subtraction and source detection

In [ ]:
show_image(img, figsize=(10, 10), log=False, dpi=50)

In [ ]:
img_nobkg = swap_byteorder(img.data)

In [ ]:
objects = sep.extract(img_nobkg, 2, err=bkg.globalrms)

In [ ]:
plot_sources(objects, img_nobkg)


# Magnitude calculation

In [ ]:
ap_radii_pix

In [ ]:
fluxes = []
fluxerrs = []
flags = []
ap_radii = np.array([0.5, 1, 2, 3, 5, 10, 15, 20])
ap_radii_pix = ap_radii / plate_scale

for r in ap_radii_pix:
    flux, fluxerr, flag = sep.sum_circle(img_nobkg, objects['x'], objects['y'],
                                         r, err=np.median(bkg_rms), gain=1.0)

    fluxes.append(flux / exptime * gain)
    fluxerrs.append(fluxerr / exptime * gain)
    flags.append(flag)

In [ ]:
mags_magerrs = [to_mag(flux, flux_err, std_data["zeropoint"], atm_ext) for flux, flux_err in zip(fluxes, fluxerrs)]
mags = [mag[0] for mag in mags_magerrs]
magerrs = [mag[1] for mag in mags_magerrs]

In [ ]:
r_half, flags_r_half = sep.flux_radius(img_nobkg, objects["x"], objects["y"], np.full(len(objects), 100.), 0.5)

In [ ]:
r_kron, flags_r_kron = sep.kron_radius(img_nobkg, objects["x"], objects["y"], objects["a"], objects["b"], objects["theta"], 20/plate_scale)

In [ ]:
img.header

In [ ]:
plt.figure()

hm = arya.HueMap((0, len(ap_radii)))
for i in range(len(ap_radii)):
    plt.scatter(mags[i], magerrs[i], alpha=0.1, c=hm(i))

plt.xlabel("magnitude " + filtname)
plt.ylabel("magnitude error")

plt.xlim(20, 30)
plt.ylim(0, 1)
plt.scatter(sigma5_depth, 0.2, color="red", s=5)
arya.Colorbar(hm, ax=plt.gca(), label="apeture / arcmin")
plt.tight_layout()

In [ ]:
plt.figure()
plt.hist(magerrs, bins=np.linspace(0, 1, 30))

In [ ]:
from matplotlib import cm

In [ ]:
log_r_h = np.log10(r_kron)

In [ ]:
log_r_range = np.min(log_r_h[log_r_h > 0]), np.max(log_r_h)

In [ ]:
plt.figure()
plt.hist(r_kron, 100)


In [ ]:
filt_good = (10 < mags[1]) & (mags[1] < 18)
filt_nonext = (r_half > 0.5) & (r_half < 10)

In [ ]:
coords = img.wcs.pixel_to_world(objects["x"], objects["y"])

In [ ]:
catalogue = Table(
    {"ra": coords.ra,
    "dec": coords.dec,
    "ellipse_a": objects["a"],
    "ellipse_b": objects["b"],
     "r_half": r_half,
    },
)


for i in range(len(ap_radii)):
    catalogue[f"flux_{i}"] = fluxes[i]
    catalogue[f"flux_err_{i}"] = fluxerrs[i]

    catalogue[f"mag_{i}"] = mags[i]
    catalogue[f"mag_err_{i}"] = magerrs[i]
    

catalogue

In [ ]:
from pathlib import Path
import tomli_w

In [ ]:
dirname_out = Path(
    str(Path(filename).parent).replace("imaging", "photometry").replace("reduced/object", "sep")
)


filename_notes =  dirname_out/ (Path(filename).stem + "-photom.toml")
filename_catalogue = dirname_out / (Path(filename).stem + "-cat.fits")

filename_notes,
filename_catalogue

In [ ]:
dirname_out.mkdir(parents=True, exist_ok=True)

In [ ]:
catalogue.write(filename_catalogue)